In [1]:
import polars as pl
from great_tables import GT
import numpy as np
import gc

In [2]:
import altair as alt
import plotly.express as px

In [3]:
grocery_df = pl.scan_csv(source="data/panel-datasets/edible_grocery.csv", 
                         has_header=True, 
                         separator=",", 
                         schema_overrides={'panel_id': pl.Int32,
                                           'trans_id': pl.Int32,
                                           'week': pl.Int16,
                                           'sku_id': pl.Categorical,
                                           'units': pl.Int16,
                                           'price': pl.Float32,
                                           'brand': pl.Categorical})

In [4]:
grocery_df = pl.scan_csv(source="data/panel-datasets/sku_weight.csv", 
                         has_header=True, 
                         separator=",", 
                         schema_overrides={'sku_id': pl.Int32,
                                           'weight': pl.Int16})

In [5]:
kiwi_df = pl.scan_csv(source="data/panel-datasets/kiwibubbles_trans.csv", 
                      has_header=True, 
                      separator=",", 
                      schema_overrides={'ID': pl.Int16,
                                        'Market': pl.Categorical,
                                        'Day': pl.Int16,
                                        'Units': pl.Int16})